# Building a Simple GIS with Yelp API and Folium - Lab

## Introduction 

So we have learned quite a bit about APIs and how they are now big buzzword in the tech industry. Think of it as a protocol for how to make requests and communicate with another server. We have seen how to mine Twitter for getting text data and apply basic frequency based NLP techniques to get some insight. 

One of the key aspects of being a data scientist is the ability to learn how a new API works, how to go through its specific authentication process (OAuth) and how to process the data structures that get returned as a response to our requests. It is a good practice to spend some time learning the API through the official documentation before sending in requests. 

On these lines, this lab requires you to learn another popular API (YELP Fusion) by following the provided detailed online documentation. We shall build a simple Geographical Information System (GIS) using the data from yelp.

## Objectives
You will be able to: 
* Successfully sign up for Yelp API 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Create a simple geographical system on to view information about selected businesses, at a given location. 


## The Yelp Fusion API - v3


### Point your browser over to this [yelp page](https://www.yelp.com/developers/v3/manage_app) and create an app in order to obtain  `client_id` and `api_key` tokens. 

**NOTE:** You will be required to sign up using Google or Facebook etc. if you dont already have an account.

<img src="yelp_app.png" width=500>



After registration, you'll be presented with your account information and limits of your access. For Yelp, or any other API for that reason, you need to make sure that you dont surpass your request quota, otherwise, you may end up getting banned in some cases. Yelp shows this information to you as below:

<img src="quota.png" width=500>

### Save your api_key and client_id in the variables below:

In [1]:
import operator 
import json
from collections import Counter
import pandas as pd
import nltk
from nltk import bigrams

In [2]:
with open('') as f:
    d = json.load(f)
    client_id = d['client_id']
    api_key = d['api_key']
    

## The `yelpapi` 

The yelpapi is a pure Python implementation of the Yelp Fusion API (aka Yelp v3 API). It is simple, fast, and robust to any changes Yelp may make to the API in the future. See tha basic usage of this library on the [official Github repo](https://github.com/gfairchild/yelpapi). You may look out for other APIs to achieve this but for this lesson, we shall use it for sake of simplicity. 

First you must pip install the library

In [3]:
#!pip install yelpapi

### Import `yelpapi ` into working environment and pass in the api_key as shown in the Github Repo 

In [4]:
# Code here
from yelpapi import YelpAPI
yelp_api = YelpAPI(api_key, timeout_s=3.0)


### The Api request and response

Great so we can now start making API calls using the format:
```python
response = yelp_api.search_query(term =<search term>, 
                                 location=<search location>, 
                                 sort_by='rating', 
                                 limit=50)
```
We can pass in a lot more arguments to refine our search. [Here is a complete list of options that search API provides us](https://www.yelp.com/developers/documentation/v3/business_search)

* Make an API request using a simple criteria location and term
* save the response as `response` 
* inspect the type and contents of `response`.

In [5]:
## Pass in a spcific term and location to make a call. 
#response = yelp_api.search_query(term =<term>,
#                               location=<location>)
# For this example, we are looking for chinese food in London.

term = 'Spur'
location = 'Australia'

In [6]:
# Make an API call using chosen term and location
response = yelp_api.search_query(term=term, location=location, sort_by='rating', limit=40)
type(response)
print(response)



{'businesses': [{'id': 'mHOjAxa9S8eBLXKPiOupZg', 'alias': 'silver-spur-steak-ranch-penrith', 'name': 'Silver Spur Steak Ranch', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/YMz5f-9KF8HwktODKJCglQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/silver-spur-steak-ranch-penrith?adjust_creative=5e9xC0TBE581R5tL4N2iKg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=5e9xC0TBE581R5tL4N2iKg', 'review_count': 5, 'categories': [{'alias': 'steak', 'title': 'Steakhouses'}], 'rating': 4.5, 'coordinates': {'latitude': -33.7598104065338, 'longitude': 150.684002253079}, 'transactions': [], 'price': '$$', 'location': {'address1': '123 Mulgoa Rd', 'address2': None, 'address3': None, 'city': 'Penrith', 'zip_code': '2750', 'country': 'AU', 'state': 'NSW', 'display_address': ['123 Mulgoa Rd', 'Penrith New South Wales 2750', 'Australia']}, 'phone': '+61247321700', 'display_phone': '+61 2 4732 1700', 'distance': 49098.16984607218}, {'id': 'eItrrZqEyl23w005JFiIHg', 

### JSON .. again ! 

We have a nice nifty little return now! As you can see, the contents of the response is formatted as a string but what kind of data structures does this remind you of?  

To start there's the outer curly brackets:  

#### {"businesses":   

Hopefully you're thinking 'hey that's just like a python dictionary!'

Then within that we have what appears to be a list of dictionaries:  

#### {"id": "jeWIYbgBho9vBDhc5S1xvg",

This response is an example of a JSON (JavaScript Object Notation) format that we've seen so many times before. We can simply treat it as a dictionary and process it further. 

### Inspect the values for all the keys in the response

In [7]:
# inspect the key value pairs to understand the strcuture of data 
for key in response.keys():
    print(type(key))



<class 'str'>
<class 'str'>
<class 'str'>


Whoops, what's going on here!? Well, notice from our previous preview of the response that we saw there were a hierarhcy within the response. Let's begin to investigate further to see what the problem is.

First, recall that the overall strucutre of the response was a dictionary. Let's look at what the keys are:

In [8]:
response.keys()

dict_keys(['businesses', 'total', 'region'])

Consult the Yelp API and learn what value is carried in each key. 

#### Continue to preview these keys  further to get a little better acquainted. 

In [9]:
print('BUSINESS:', response['businesses'][0])
print(" ")
print('REGION:', response['region'])
print(" ")
print('TOTAL :',response['total'])

BUSINESS: {'id': 'mHOjAxa9S8eBLXKPiOupZg', 'alias': 'silver-spur-steak-ranch-penrith', 'name': 'Silver Spur Steak Ranch', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/YMz5f-9KF8HwktODKJCglQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/silver-spur-steak-ranch-penrith?adjust_creative=5e9xC0TBE581R5tL4N2iKg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=5e9xC0TBE581R5tL4N2iKg', 'review_count': 5, 'categories': [{'alias': 'steak', 'title': 'Steakhouses'}], 'rating': 4.5, 'coordinates': {'latitude': -33.7598104065338, 'longitude': 150.684002253079}, 'transactions': [], 'price': '$$', 'location': {'address1': '123 Mulgoa Rd', 'address2': None, 'address3': None, 'city': 'Penrith', 'zip_code': '2750', 'country': 'AU', 'state': 'NSW', 'display_address': ['123 Mulgoa Rd', 'Penrith New South Wales 2750', 'Australia']}, 'phone': '+61247321700', 'display_phone': '+61 2 4732 1700', 'distance': 49098.16984607218}
 
REGION: {'center': {'longitude': 151.1

This makes more sense, so we are mainly interested in the `businesses` for our needs. 

### Print the names of businesses and included ratings 

In [10]:
# Code here 
for business in response['businesses']:
    print("Name: {}".format (business['name']))
    print("Rating: {}".format (business['rating']))

Name: Silver Spur Steak Ranch
Rating: 4.5
Name: Mustang Spur
Rating: 1.0
Name: Spur
Rating: 1.0
Name: Sydney Opera House
Rating: 4.5
Name: The Bank Hotel
Rating: 3.5
Name: Fish on Fire
Rating: 4.0
Name: Clover Cafe
Rating: 3.5
Name: Corellis Cafe Gallery
Rating: 3.0
Name: Mad Pizza e Bar
Rating: 3.5


Great, now are are getting somewhere. It is a good idea at this stage to store this information as a dataframe for processing further. 
### Create a Pandas dataframe for contents of `businesses`
* Check the number of records in the dataframe
* Inspect the columns and head

In [14]:
# Code here 
spur_aus = pd.DataFrame.from_dict(response['businesses'])
spur_aus.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,silver-spur-steak-ranch-penrith,"[{'alias': 'steak', 'title': 'Steakhouses'}]","{'latitude': -33.7598104065338, 'longitude': 1...",+61 2 4732 1700,49098.169846,mHOjAxa9S8eBLXKPiOupZg,https://s3-media3.fl.yelpcdn.com/bphoto/YMz5f-...,False,"{'address1': '123 Mulgoa Rd', 'address2': None...",Silver Spur Steak Ranch,+61247321700,$$,4.5,5,[],https://www.yelp.com/biz/silver-spur-steak-ran...
1,mustang-spur-campbelltown-4,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...","{'latitude': -34.073042, 'longitude': 150.797335}",+61 2 4627 1255,43081.564367,eItrrZqEyl23w005JFiIHg,https://s3-media1.fl.yelpcdn.com/bphoto/xfrOV0...,False,{'address1': '38 Level 1 Kellicar Rd Campbellt...,Mustang Spur,+61246271255,$$$,1.0,3,[],https://www.yelp.com/biz/mustang-spur-campbell...
2,spur-campbelltown,"[{'alias': 'steak', 'title': 'Steakhouses'}]","{'latitude': -34.076943, 'longitude': 150.790614}",+61 2 4627 1255,43893.738316,JOES9r9YK2kFPF7JdW6TJQ,,False,"{'address1': 'Shop 37/38', 'address2': 'Level ...",Spur,+61246271255,NaN,1.0,1,[],https://www.yelp.com/biz/spur-campbelltown?adj...
3,sydney-opera-house-sydney,"[{'alias': 'venues', 'title': 'Venues & Event ...","{'latitude': -33.857123, 'longitude': 151.215016}",+61 2 9250 7111,2407.580264,xYiGsPaQ0K5_Eut_kK7uMA,https://s3-media2.fl.yelpcdn.com/bphoto/DTZHKV...,False,"{'address1': 'Bennelong Point', 'address2': ''...",Sydney Opera House,+61292507111,NaN,4.5,269,[],https://www.yelp.com/biz/sydney-opera-house-sy...
4,the-bank-hotel-newtown,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...","{'latitude': -33.897591, 'longitude': 151.178909}",+61 2 8568 1900,3210.439345,hWWTjpSIAjXCdSrwrEKydw,https://s3-media3.fl.yelpcdn.com/bphoto/roy566...,False,"{'address1': '324 King St', 'address2': None, ...",The Bank Hotel,+61285681900,$$,3.5,27,[],https://www.yelp.com/biz/the-bank-hotel-newtow...


This is fantastic. We have successfully learned a new API , made requests to it, recieved and studied the response and stored the results in a dataframe and can now enjoy all the goodness of Pandas. Thats quite a bit of data engineering. 

### Visualize the location from search query
The `region` key in the response carries the geographical information for the region searched.
* Get the latitude / longitude information from `region`
* Create a folium map with these coordinates. 
* Use a zoom start value = 13

In [42]:
import folium

lat_long = response['region']['center']
lat = lat_long['latitude']
long = lat_long['longitude']

aus_map = folium.Map([lat, long], zoom_start=8)
aus_map
# Code here

Expected Output:
![](london.png)

Nice. We can now extract the coordinate information for each business and plot it on this map.

### Get the business coordinates from dataframe for each business and plot on the map above

In [57]:
# Code here 
#coordinates = []
for business in response['businesses']:
    lat = business['coordinates']['latitude']
    #print(lat)
    long = business['coordinates']['longitude']


    marker = folium.Marker(location=[lat, long])
    marker.add_to(aus_map)
aus_map

Expected output:
![](markers.png)

Kool so we have everything in place but the visualization is still not very *Informative* so to speak. You can't tell which marker represents which business and also other information on business like rating, cost, links to user reviews etc. is still not visible. SO its geographical , but not exactly an Information System yet as you cant make any decisions on this information.  Here's as example of what it possible can look like
![](out.png)


For this you need to understand `folium.popup()` which let's you click on a marker to show a pop up window. This window acts more like an HTML page so you can easily format the information you present in the popup using following values:
* The official business logo/image:  `image_url`
* Name of the Business: `name`
* Price (how expensive): `price`
* Links to user reviews on Yelp: `url`

Doing this in HTML is not required , so we recommend that you first try to put in basic information in the popups as just text. As a next stage , you can start changing into HTML code to make it visually more appealing.

### Attempt to recreate the interactive visualization shown above.

Here's a good resource with code examples on [how to create folium popups](https://github.com/python-visualization/folium/blob/master/examples/Popups.ipynb)

In [63]:
from folium import IFrame

for row in spur_aus.index:
    
    lat_long = spur_aus['coordinates'][row]
    lat = lat_long['latitude']
    long = lat_long['longitude']

    name = 'Business Name: ' + str(spur_aus['name'][row])
    rating = 'Rating: ' + str(spur_aus['rating'][row])
    price = 'Price: ' + str(spur_aus['price'][row])

    img = spur_aus['image_url'][row]
    txt = folium.Html('<head><style> body {background-image: url('+ img+ ');\
                        background-color: rgba(255,255,255,0.6);background-blend-mode: lighten;\
                        background-position: center; background-repeat: no-repeat; background-size: cover;}</style></head>' +
                      '<H2><center><font color="black">' + str(spur_aus['name'][row]) + '</H2></center>' +\
                      '<B><center>Rating: </B>' + str(spur_aus['rating'][row]) + '/ 5</br></center>' +\
                      '<B><center>Price: </B>' + str(spur_aus['price'][row]) + '</br></center>' +\
                      '<center><a href='+ str(spur_aus['url'][row])+ ' target="_blank">Read Reviews</b></a></br></br></center>' + \
                      '<small>Searching for '+ term + ' in ' + location, script=True)

    iframe = folium.IFrame(txt, width=400, height=150)
    popup = folium.Popup(iframe, max_width=3000)
    Text = folium.Marker(location=[lat,long], popup=popup,icon=folium.Icon(icon_color='green'))
    aus_map.add_child(Text)

aus_map

Wow . An Interactive Geographical information System backed by live data through API calling. 

<img src="star.jpg" width=300>

### More APIs to Checkout

* Google Maps
* Twitter
* AWS
* IBM's Watson
* Yelp

## Summary 

In this lab, we learned how to use the Yelp API with authentication, making calls, understanding the responses and creating interactive geographical visualizations in Folium. We encourage you to re-visit this lab again once you have studied some important machine learning algorithms to make predictions , find similarities, group/cluster businesses or classify them based on user criteria. 